In [32]:
from glob import glob
from tqdm import tqdm
import json

In [3]:
!ls

agent.py		      prepare-openhermes.ipynb
api.py			      __pycache__
code-feedback		      python-00000-of-00001.jsonl
Code-Feedback.jsonl	      pythonanalytics-00000-of-00001.jsonl
delete-unhealthy.sh	      python-code-feedback.jsonl
docker-compose.yaml	      python-openhermes.jsonl
generate-code-feedback.ipynb  README.md
generate-openhermes.ipynb     requirements.txt
jupyter.py		      run.ipynb
openhermes		      Untitled.ipynb
openhermes2_5.json	      upload-code-feedback.ipynb
prepared-openhermes.json      upload-openhermes.ipynb


In [4]:
files = glob('code-feedback/*.json')
files.extend(glob('openhermes/*.json'))
len(files)

118390

In [26]:
texts = set()

for f in files:
    with open(f) as fopen:
        data = json.load(fopen)

    if data['result'] is None:
        continue
    if len(data['result']) == 3:
        continue
    if 'I have reached the maximum number of executions' in data['result'][-1]['content']:
        continue
    texts.add(data['result'][1]['content'])
    for d in data['result'][2:]:
        if d['role'] == 'user':
            continue
        else:
            texts.add(d['content'])
len(texts)

241971

In [30]:
!nvidia-smi

Sun Mar 31 13:47:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     On  | 00000000:01:00.0 Off |                  Off |
| 30%   42C    P5             114W / 350W |      6MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [31]:
with open('/home/husein/ssd1/ctranslate2/python-act.texts', 'w') as fopen:
    for t in texts:
        if not len(t):
            continue
        fopen.write(f'{json.dumps(t)}\n')

In [33]:
files = glob('/home/husein/ssd1/ctranslate2/python-act/*.json')
len(files)

241970

In [34]:
filtered, accepted = [], {}
for f in tqdm(files):
    with open(f) as fopen:
        r = json.load(fopen)
    try:
        splitted = r['r'].split()
        ratio = (len(set(splitted)) / len(splitted))
        if ratio < 0.5:
            filtered.append((r, ratio))
        else:
            accepted[r['src']] = r['r']
    except:
        filtered.append((r, 0))
        
len(accepted), len(filtered)

100%|██████████████████████████████████████████████████████████████████████| 241970/241970 [00:04<00:00, 56717.83it/s]


(231381, 10589)

In [35]:
files = glob('code-feedback/*.json')
files.extend(glob('openhermes/*.json'))
len(files)

118390

In [36]:
with open(files[0]) as fopen:
    data = json.load(fopen)

In [38]:
data.keys()

dict_keys(['original', 'result'])

In [55]:
with open('translated-combined.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)

        if data['result'] is None:
            continue
        if len(data['result']) == 3:
            continue
        if 'I have reached the maximum number of executions' in data['result'][-1]['content']:
            continue

        for i in range(len(data['result'])):
            data['result'][i]['content_ms'] = accepted.get(data['result'][i]['content'])
        
        fopen_l.write(f'{json.dumps(data)}\n')

100%|██████████████████████████████████████████████████████████████████████| 118390/118390 [00:05<00:00, 20960.71it/s]


In [1]:
from huggingface_hub import HfApi
api = HfApi()

In [2]:
api.upload_file(
    path_or_fileobj='translated-combined.jsonl',
    path_in_repo='translated-combined.jsonl',
    repo_id='mesolitica/python-act',
    repo_type='dataset',
)

translated-combined.jsonl:   0%|          | 0.00/591M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/python-act/commit/feb1518ff48f72b3fd72f4542f5d36eff277dc2d', commit_message='Upload translated-combined.jsonl with huggingface_hub', commit_description='', oid='feb1518ff48f72b3fd72f4542f5d36eff277dc2d', pr_url=None, pr_revision=None, pr_num=None)